In [3]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
print('Done')

Done


In [4]:
print(np.__version__)

1.18.5


In [23]:
train = pd.read_csv('Tweettrain.csv')

In [24]:
train.head(15)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [25]:
train = train[['text','sentiment']]

In [26]:
train["text"].isnull().sum()

1

In [27]:
train["text"].fillna("No content", inplace = True)

In [28]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [29]:
temp = []
#Splitting pd.Series to list
data_to_list = train['text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

[' I`d have responded, if I were going',
 ' Sooo SAD I will miss you here in San Diego!!!',
 'my boss is bullying me...',
 ' what interview! leave me alone',
 ' Sons of ****, why couldn`t they put them on the releases we already bought']

In [30]:
def sent_to_words(sentences):
    for sentence in sentences:
        nopunct = re.sub(r'[^\w\s]','',sentence)
        yield(nopunct.lower().split())  
        

data_words = list(sent_to_words(temp))

In [31]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [32]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [33]:
data = np.array(data)

In [34]:
labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'negative':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y

In [35]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    1  119   45]
 [   0    0    0 ...   10 1462 2229]
 [   0    0    0 ... 1300    9   15]
 ...
 [   0    0    0 ...  631  874 2747]
 [   0    0    0 ...   27  686    8]
 [   0    0    0 ... 2230  224  666]]


In [36]:
print(labels)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [37]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

20610 6871 20610 6871


In [38]:
import numpy
numpy.version.version

'1.18.5'

In [39]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(3,activation='softmax'))

model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
645/645 [==============================] - ETA: 0s - loss: 0.9734 - accuracy: 0.5036
Epoch 00001: val_accuracy improved from -inf to 0.61054, saving model to best_model1.hdf5
645/645 [==============================] - 34s 53ms/step - loss: 0.9734 - accuracy: 0.5036 - val_loss: 0.8373 - val_accuracy: 0.6105
Epoch 2/70
645/645 [==============================] - ETA: 0s - loss: 0.7853 - accuracy: 0.6492
Epoch 00002: val_accuracy improved from 0.61054 to 0.67850, saving model to best_model1.hdf5
645/645 [==============================] - 30s 47ms/step - loss: 0.7853 - accuracy: 0.6492 - val_loss: 0.7519 - val_accuracy: 0.6785
Epoch 3/70
644/645 [============================>.] - ETA: 0s - loss: 0.7252 - accuracy: 0.6925
Epoch 00003: val_accuracy improved from 0.67850 to 0.68229, saving model to best_model1.hdf5
645/645 [==============================] - 30s 47ms/step - loss: 0.7253 - accuracy: 0.6925 - val_loss: 0.7308 - val_accuracy: 0.6823
Epoch 4/70
645/645 [=================

Epoch 27/70
644/645 [============================>.] - ETA: 0s - loss: 0.5385 - accuracy: 0.7889
Epoch 00027: val_accuracy did not improve from 0.73002
645/645 [==============================] - 30s 47ms/step - loss: 0.5385 - accuracy: 0.7889 - val_loss: 0.6764 - val_accuracy: 0.7274
Epoch 28/70
645/645 [==============================] - ETA: 0s - loss: 0.5311 - accuracy: 0.7951
Epoch 00028: val_accuracy did not improve from 0.73002
645/645 [==============================] - 30s 47ms/step - loss: 0.5311 - accuracy: 0.7951 - val_loss: 0.6818 - val_accuracy: 0.7248
Epoch 29/70
645/645 [==============================] - ETA: 0s - loss: 0.5331 - accuracy: 0.7947
Epoch 00029: val_accuracy did not improve from 0.73002
645/645 [==============================] - 30s 47ms/step - loss: 0.5331 - accuracy: 0.7947 - val_loss: 0.6867 - val_accuracy: 0.7281
Epoch 30/70
645/645 [==============================] - ETA: 0s - loss: 0.5287 - accuracy: 0.7933
Epoch 00030: val_accuracy did not improve from 0

Epoch 56/70
644/645 [============================>.] - ETA: 0s - loss: 0.4930 - accuracy: 0.8103
Epoch 00056: val_accuracy did not improve from 0.73075
645/645 [==============================] - 30s 47ms/step - loss: 0.4930 - accuracy: 0.8103 - val_loss: 0.7142 - val_accuracy: 0.7245
Epoch 57/70
644/645 [============================>.] - ETA: 0s - loss: 0.4860 - accuracy: 0.8136
Epoch 00057: val_accuracy did not improve from 0.73075
645/645 [==============================] - 30s 47ms/step - loss: 0.4861 - accuracy: 0.8135 - val_loss: 0.7117 - val_accuracy: 0.7207
Epoch 58/70
644/645 [============================>.] - ETA: 0s - loss: 0.4837 - accuracy: 0.8161
Epoch 00058: val_accuracy did not improve from 0.73075
645/645 [==============================] - 30s 47ms/step - loss: 0.4837 - accuracy: 0.8162 - val_loss: 0.7215 - val_accuracy: 0.7191
Epoch 59/70
644/645 [============================>.] - ETA: 0s - loss: 0.4874 - accuracy: 0.8126
Epoch 00059: val_accuracy did not improve from 0

In [40]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/70
645/645 [==============================] - ETA: 0s - loss: 0.9531 - accuracy: 0.5227
Epoch 00001: val_accuracy improved from -inf to 0.59322, saving model to best_model2.hdf5
645/645 [==============================] - 49s 75ms/step - loss: 0.9531 - accuracy: 0.5227 - val_loss: 0.8529 - val_accuracy: 0.5932
Epoch 2/70
645/645 [==============================] - ETA: 0s - loss: 0.7748 - accuracy: 0.6613
Epoch 00002: val_accuracy improved from 0.59322 to 0.67239, saving model to best_model2.hdf5
645/645 [==============================] - 39s 61ms/step - loss: 0.7748 - accuracy: 0.6613 - val_loss: 0.7538 - val_accuracy: 0.6724
Epoch 3/70
645/645 [==============================] - ETA: 0s - loss: 0.7153 - accuracy: 0.6966
Epoch 00003: val_accuracy improved from 0.67239 to 0.69218, saving model to best_model2.hdf5
645/645 [==============================] - 38s 59ms/step - loss: 0.7153 - accuracy: 0.6966 - val_loss: 0.7212 - val_accuracy: 0.6922
Epoch 4/70
645/645 [=================

Epoch 28/70
645/645 [==============================] - ETA: 0s - loss: 0.5222 - accuracy: 0.8001
Epoch 00028: val_accuracy did not improve from 0.72857
645/645 [==============================] - 40s 61ms/step - loss: 0.5222 - accuracy: 0.8001 - val_loss: 0.7023 - val_accuracy: 0.7261
Epoch 29/70
645/645 [==============================] - ETA: 0s - loss: 0.5189 - accuracy: 0.8012
Epoch 00029: val_accuracy did not improve from 0.72857
645/645 [==============================] - 39s 61ms/step - loss: 0.5189 - accuracy: 0.8012 - val_loss: 0.6872 - val_accuracy: 0.7252
Epoch 30/70
645/645 [==============================] - ETA: 0s - loss: 0.5178 - accuracy: 0.7989
Epoch 00030: val_accuracy did not improve from 0.72857
645/645 [==============================] - 39s 61ms/step - loss: 0.5178 - accuracy: 0.7989 - val_loss: 0.7087 - val_accuracy: 0.7223
Epoch 31/70
645/645 [==============================] - ETA: 0s - loss: 0.5139 - accuracy: 0.8030
Epoch 00031: val_accuracy did not improve from 0

Epoch 57/70
645/645 [==============================] - ETA: 0s - loss: 0.4649 - accuracy: 0.8247
Epoch 00057: val_accuracy did not improve from 0.72988
645/645 [==============================] - 39s 60ms/step - loss: 0.4649 - accuracy: 0.8247 - val_loss: 0.7326 - val_accuracy: 0.7115
Epoch 58/70
645/645 [==============================] - ETA: 0s - loss: 0.4641 - accuracy: 0.8242
Epoch 00058: val_accuracy did not improve from 0.72988
645/645 [==============================] - 39s 60ms/step - loss: 0.4641 - accuracy: 0.8242 - val_loss: 0.7413 - val_accuracy: 0.7241
Epoch 59/70
645/645 [==============================] - ETA: 0s - loss: 0.4615 - accuracy: 0.8282
Epoch 00059: val_accuracy did not improve from 0.72988
645/645 [==============================] - 39s 60ms/step - loss: 0.4615 - accuracy: 0.8282 - val_loss: 0.7421 - val_accuracy: 0.7149
Epoch 60/70
645/645 [==============================] - ETA: 0s - loss: 0.4570 - accuracy: 0.8254
Epoch 00060: val_accuracy did not improve from 0

In [41]:
best_model = keras.models.load_model("best_model2.hdf5")

In [42]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

215/215 - 2s - loss: 0.6942 - accuracy: 0.7299
Model accuracy:  0.7298792004585266


In [43]:
predictions = best_model.predict(X_test)

In [64]:
train2 = pd.read_csv('stockerbot-export.csv',error_bad_lines=False)

b'Skipping line 3190: expected 8 fields, saw 17\nSkipping line 3201: expected 8 fields, saw 17\nSkipping line 3251: expected 8 fields, saw 17\nSkipping line 3516: expected 8 fields, saw 17\nSkipping line 4074: expected 8 fields, saw 17\nSkipping line 4083: expected 8 fields, saw 17\nSkipping line 4084: expected 8 fields, saw 17\nSkipping line 4495: expected 8 fields, saw 12\n'


In [67]:
train2.shape

(28264, 8)

In [71]:
train2 = train2[['text']]
train2["text"].isnull().sum()

0

In [72]:
temp = []
data_to_list = train2['text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))

In [73]:
data_words = list(sent_to_words(temp))

In [74]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [76]:
train2['text']=data

In [82]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

new_words = {
    'foo': 2.0,
    'bar': -3.4,
}

SIA = SentimentIntensityAnalyzer()

SIA.lexicon.update(new_words)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Alkou\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [84]:
def findpolarity(data):
    sid = SentimentIntensityAnalyzer()
    polarity = sid.polarity_scores(data)
    if(polarity['compound'] >= 0.2):  
        sentiment = 'positive'
    if(polarity['compound'] <= -0.2):
        sentiment = 'negative'
    if(polarity['compound'] < 0.2 and polarity['compound'] >-0.2):
        sentiment = 'neutral'     
    return(sentiment)

sentiment=[]
for tweet in train2['text']:
    s = findpolarity(tweet)
    sentiment.append(s)

In [88]:
len(sentiment)

28264

In [89]:
train2['sentiment']=sentiment

In [136]:
train2.to_csv('tweet_sentiment_train.csv', index=False)

In [137]:
#we have created our new financial tweet dataset and are going to train our model on that one
train = pd.read_csv('tweet_sentiment_train.csv')

In [154]:
temp = []
#Splitting pd.Series to list
data_to_list = train['text'].values.tolist()

In [155]:
data = np.array(data_to_list)

In [141]:
labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'negative':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y

In [142]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  381  381 3022]
 [   0    0    0 ...    9  346 3766]
 [   0    0    0 ...  315    9 3767]
 ...
 [   0    0    0 ...  273  133  327]
 [   0    0    0 ... 4466 4467  133]
 [   0    0    0 ...    2  815  157]]


In [100]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(tweets,labels, random_state=0)
print (len(X_train2),len(X_test2),len(y_train2),len(y_test2))

21198 7066 21198 7066


In [102]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train2, y_train2, epochs=70,validation_data=(X_test2, y_test2),callbacks=[checkpoint2])

Epoch 1/70
663/663 [==============================] - ETA: 0s - loss: 0.6577 - accuracy: 0.7312
Epoch 00001: val_accuracy improved from -inf to 0.80512, saving model to best_model2.hdf5
663/663 [==============================] - 51s 76ms/step - loss: 0.6577 - accuracy: 0.7312 - val_loss: 0.5027 - val_accuracy: 0.8051
Epoch 2/70
663/663 [==============================] - ETA: 0s - loss: 0.4193 - accuracy: 0.8471
Epoch 00002: val_accuracy improved from 0.80512 to 0.87263, saving model to best_model2.hdf5
663/663 [==============================] - 48s 73ms/step - loss: 0.4193 - accuracy: 0.8471 - val_loss: 0.3577 - val_accuracy: 0.8726
Epoch 3/70
663/663 [==============================] - ETA: 0s - loss: 0.3258 - accuracy: 0.8875
Epoch 00003: val_accuracy improved from 0.87263 to 0.90051, saving model to best_model2.hdf5
663/663 [==============================] - 49s 73ms/step - loss: 0.3258 - accuracy: 0.8875 - val_loss: 0.2993 - val_accuracy: 0.9005
Epoch 4/70
663/663 [=================

Epoch 27/70
663/663 [==============================] - ETA: 0s - loss: 0.1523 - accuracy: 0.9517
Epoch 00027: val_accuracy improved from 0.94198 to 0.94523, saving model to best_model2.hdf5
663/663 [==============================] - 48s 72ms/step - loss: 0.1523 - accuracy: 0.9517 - val_loss: 0.1990 - val_accuracy: 0.9452
Epoch 28/70
663/663 [==============================] - ETA: 0s - loss: 0.1515 - accuracy: 0.9509
Epoch 00028: val_accuracy did not improve from 0.94523
663/663 [==============================] - 48s 72ms/step - loss: 0.1515 - accuracy: 0.9509 - val_loss: 0.1922 - val_accuracy: 0.9449
Epoch 29/70
663/663 [==============================] - ETA: 0s - loss: 0.1453 - accuracy: 0.9531
Epoch 00029: val_accuracy improved from 0.94523 to 0.94551, saving model to best_model2.hdf5
663/663 [==============================] - 48s 72ms/step - loss: 0.1453 - accuracy: 0.9531 - val_loss: 0.1968 - val_accuracy: 0.9455
Epoch 30/70
663/663 [==============================] - ETA: 0s - loss

Epoch 55/70
663/663 [==============================] - ETA: 0s - loss: 0.1318 - accuracy: 0.9587
Epoch 00055: val_accuracy did not improve from 0.94750
663/663 [==============================] - 47s 70ms/step - loss: 0.1318 - accuracy: 0.9587 - val_loss: 0.1996 - val_accuracy: 0.9449
Epoch 56/70
663/663 [==============================] - ETA: 0s - loss: 0.1319 - accuracy: 0.9571
Epoch 00056: val_accuracy improved from 0.94750 to 0.94792, saving model to best_model2.hdf5
663/663 [==============================] - 47s 70ms/step - loss: 0.1319 - accuracy: 0.9571 - val_loss: 0.1945 - val_accuracy: 0.9479
Epoch 57/70
663/663 [==============================] - ETA: 0s - loss: 0.1321 - accuracy: 0.9582
Epoch 00057: val_accuracy did not improve from 0.94792
663/663 [==============================] - 46s 70ms/step - loss: 0.1321 - accuracy: 0.9582 - val_loss: 0.1908 - val_accuracy: 0.9472
Epoch 58/70
663/663 [==============================] - ETA: 0s - loss: 0.1270 - accuracy: 0.9595
Epoch 0005

In [143]:
best_model = keras.models.load_model("best_model2.hdf5")

In [160]:
test_loss, test_acc = best_model.evaluate(X_test2, y_test2, verbose=2)
print('Model accuracy: ',test_acc)

221/221 - 2s - loss: 0.1902 - accuracy: 0.9492
7066
Model accuracy:  0.9491932988166809


In [153]:
sequence = tokenizer.texts_to_sequences(['BTC is going to decrease in value over the next three days'])
test = pad_sequences(sequence, maxlen=max_len)
print(test)
sentiment[np.around(best_model.predict(test), decimals=0).argmax(axis=1)[0]]

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0  133    9  406    1 1904   10  1

'negative'

In [168]:
predictions = best_model.predict(X_test2)
print((y_test2[:40]))

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
